In [41]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 
device = "cuda"

In [42]:
from __future__ import print_function

import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [43]:
import import_ipynb
from Datagen import Data

In [44]:
def likeliness(y_pred, y):
    pred = torch.zeros([len(y_pred), y.shape[1]])
    for idx, unit in enumerate(torch.argmax(y_pred, axis = 1)):
        pred[idx, unit] = 1
    return accuracy_score(pred.cpu(), y.cpu())

In [45]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        self.transforms = transforms.ToTensor()
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        x = self.x[index, ...]
        y = self.y[index, ...]
        return x, y
    
    def __len__(self):
        return len(self.x)

In [46]:
def load_data(seglength, val_ratio):
    focal, focal_labels = Data('D:\EEG\FOCAL', seglength, class_num = 1)
    ige, ige_labels = Data('D:\EEG\IGE', seglength, class_num = 2)

    X_train = torch.cat((focal, ige))
    Y_train = torch.cat((focal_labels, ige_labels))

    del focal, focal_labels, ige, ige_labels

    pnes, pnes_labels = Data('D:\EEG\PNES', seglength, class_num = 3)

    X_train = torch.cat((X_train, pnes))
    Y_train = torch.cat((Y_train, pnes_labels))

    del pnes, pnes_labels

    temporal, temporal_labels = Data('D:\EEG\TLE', seglength, class_num = 4)

    X_train = torch.cat((X_train, temporal))
    Y_train = torch.cat((Y_train, temporal_labels))

    del temporal, temporal_labels

    X_train, X_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size= val_ratio, random_state=42, stratify=Y_train)
    

    return X_train, Y_train, X_val, y_val

In [47]:
def load_test_data(seglength):
    focal, focal_labels = Data('D:\TEST EEG\FOCAL', seglength, class_num = 1)
    ige, ige_labels = Data('D:\TEST EEG\IGE', seglength, class_num = 2)

    X = torch.cat((focal, ige))
    Y = torch.cat((focal_labels, ige_labels))

    del focal, focal_labels, ige, ige_labels

    pnes, pnes_labels = Data('D:\TEST EEG\PNES', seglength, class_num = 3)

    X = torch.cat((X, pnes))
    Y = torch.cat((Y, pnes_labels))

    del pnes, pnes_labels

    temporal, temporal_labels = Data('D:\TEST EEG\TLE', seglength, class_num = 4)

    X = torch.cat((X, temporal))
    Y = torch.cat((Y, temporal_labels))

    del temporal, temporal_labels

    return X, Y

In [48]:
seglength = 90000

In [49]:
#x_test, y_test = load_test_data(seglength)

In [50]:
#x_train, y_train, x_val, y_val = load_data(seglength, val_ratio = 0.1)

In [51]:

#x_train = x_train.view(-1, 1, 18, seglength)
#x_test = x_test.view(-1, 1, 18, seglength)
#x_val = x_val.view(-1, 1, 18, seglength)


In [52]:
"""
x_train = torch.rand([42583, 1, 18, 300])
y_train = torch.ones([42583, 4]) 
y_train[:, :3] = 0
x_val = torch.rand([4731, 1, 18, 300])
y_val = torch.ones([4731, 4]) 
y_val[:, :3] = 0
x_test = torch.rand([11828, 1, 18, 300])
y_test = torch.ones([11828, 4]) 
y_test[:, :3] = 0
"""

'\nx_train = torch.rand([42583, 1, 18, 300])\ny_train = torch.ones([42583, 4]) \ny_train[:, :3] = 0\nx_val = torch.rand([4731, 1, 18, 300])\ny_val = torch.ones([4731, 4]) \ny_val[:, :3] = 0\nx_test = torch.rand([11828, 1, 18, 300])\ny_test = torch.ones([11828, 4]) \ny_test[:, :3] = 0\n'

In [53]:
class LSTM_CNN_Spatial(nn.Module):
    def __init__(self, num_classes, batch_size, T, C, input_size, hidden_size,
                 num_layers, spatial_num=8, drop_out=0.5):
        super(LSTM_CNN_Spatial, self).__init__()

        self.N = batch_size
        self.T = T
        self.C = C
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.pool = 4
        self.seq_len = self.T // self.input_size
        self.fc_in = spatial_num * self.hidden_size // self.pool
        
        self._lstm = nn.LSTM(self.input_size, self.hidden_size, 
                            self.num_layers, batch_first=True)
        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, spatial_num, (self.C, 1)),
            nn.BatchNorm2d(spatial_num),
            nn.ELU(),
            nn.AvgPool2d((1, self.pool)),
            nn.Dropout(drop_out)
        )
        
        self.fc = nn.Linear(self.fc_in , num_classes)
        
    def forward(self, x):
        # input shape of x: (N, 1, C, T)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.seq_len, self.input_size)
        _x = None
        for index, lstm in enumerate(self.lstm):
            lstm_out, _ = lstm(x[:, index, :, :], None)
            tmp = lstm_out[:, -1, :]
            tmp = tmp.unsqueeze(0)
            if _x is None:
                _x = tmp
            else:
                _x = torch.cat((_x, tmp), dim=0)
        
        # (C, N, H) ===> (N, 1, C, H)   H: hidden_size
        x = _x.permute(1, 0, 2).unsqueeze(1)
        x = self.block_1(x)
        
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return probas  

In [54]:
def train(model, criterion, optimizer, data_loader, device, train_num, epochs, x_val, y_val, logged=False):
    for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        batch_size = None
        for index, data in enumerate(data_loader):
            x, y = data
            batch_size = x.shape[0] if index == 0 else batch_size
            x, y = x.to(device), y.to(device)
            
            y_pred = model(x)
            _, pred = torch.max(y_pred, 1)
            
            correct_num += np.sum(pred.cpu().numpy() == y.cpu().numpy())
            
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += float(loss.item())
            
        batch_num = train_num // batch_size
        _loss = running_loss / (batch_num + 1)
        acc = correct_num / train_num * 100
        if not logged:
            print(f'Epoch {epoch+1}/{epochs}\tTrain loss: {_loss:.4f}')
        print('---------')
        print(f"validation accuracy:{test(model,val_loader, DEVICE)}")
        print(f"test accuracy:{test(model,test_loader, DEVICE)}")
        print('--------------------------------------------')
    
    
    if not logged:
        print('Finish Training!')

In [55]:
def test(model, data_loader, device):
    holder = []
    for index, data in enumerate(data_loader):
            x, y = data
            batch_size = x.shape[0] if index == 0 else batch_size
            x, y = x.to(device), y.to(device)
            
            holder.append(likeliness(model(x), y))


    return np.average(holder)
    
    

In [56]:
torch.cuda.empty_cache()

In [57]:
epochs = 80
batch_size = 256
input_size = 5000
hidden_size = 320
num_layers = 4
spatial_num = 120
drop_out = 0.5
learning_rate = 1e-5
logged = False
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('DEVICE: ', DEVICE)
group = 1
sorted_ = True

DEVICE:  cuda:0


In [58]:
train_num, test_num = x_train.shape[0], x_test.shape[0]
    
    
train_data = MyDataset(x_train, y_train)
test_data = MyDataset(x_test, y_test)
val_data = MyDataset(x_val, y_val)
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

In [59]:
# model = LSTM(num_classes=2, input_size=64, hidden_size=256, num_layers=2)
    
# model = LSTM_CNN(num_classes=2, channels=x_train.shape[1], input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, spatial_num=spatial_num, drop_out=drop_out)
    
# model = LSTM_CNN_Half(num_classes=2, batch_size=batch_size, T=x_train.shape[-1],
#C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
#num_layers=num_layers, spatial_num=spatial_num)
    
model = LSTM_CNN_Spatial(num_classes=4, batch_size=batch_size, T=x_train.shape[-1],
                          C=x_train.shape[-2], input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, spatial_num=spatial_num)
model = torch.nn.DataParallel(model)
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
log = []
if logged:
    log.append(f'{epochs}\t{batch_size}\t{input_size}\t{hidden_size}\t'
                   f'{num_layers}\t{spatial_num}\t{drop_out}\t')
train(model, criterion, optimizer, train_loader, DEVICE,train_num, epochs, x_val, y_val, logged=False)

In [61]:
model(x_test[300].cuda())

tensor([[1.2351e-06, 1.0344e-04, 1.3771e-03, 9.9852e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)